In [3]:
from typing import (
    Annotated,
    Sequence,
    TypedDict,
    Union,
    List
)
from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from langchain_community.tools import DuckDuckGoSearchResults
from langgraph.prebuilt import ToolNode
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_ollama import ChatOllama
from langchain_core.messages import ToolMessage,SystemMessage
from langchain_core.runnables import RunnableConfig
import json
from pydantic import BaseModel, Field


class AgentState(TypedDict):
    """The state of the agent."""

    # add_messages is a reducer
    # See https://langchain-ai.github.io/langgraph/concepts/low_level/#reducers
    messages: Annotated[Sequence[BaseMessage], add_messages]
    generated_exps: dict
    exp_title:str
    reward:Union[int,float]


wrapper = DuckDuckGoSearchAPIWrapper(max_results=10)
search = DuckDuckGoSearchResults(api_wrapper=wrapper, output_format='list')

tools = [search]

model = ChatOllama(model="jacob-ebey/phi4-tools:latest",base_url="192.168.23.138:11439")
model = model.bind_tools(tools)

In [1]:
system_prompt = '''You are a Verifier AI responsible for checking the accuracy of experimental steps across various scientific and technical domains. Your task is to verify whether each step is correct by using reasonable web searches.

For each step in the experiment, you will:

Identify the key concept(s) involved.
Generate an appropriate search query to verify the correctness of the step.
Validate the step by using the search results to confirm or correct the instruction.
Your output should be structured in JSON format with the following fields:

"step": The step as given in the experiment.
"search_query": The phrase or keywords you would search on the web to verify the step.
"verification": The expected correct verification result based on technical knowledge and search results.
Be precise, factual, and ensure that the verification aligns with established scientific and technical principles.'''

In [ ]:
class verifierBase(BaseModel):
    step: str = Field(...,description="The step as given in the experiment.")
    search_query: str = Field(...,description="The phrase or keywords you would search on the web to verify the step.")
    verification: str = Field(...,description="The expected correct verification result based on technical knowledge and search results.")
class verifierStructure(BaseModel):
    steps: List[verifierBase] = Field(..., description="A list of verification steps, each containing step details, search query, and verification result.")


def generator(state:AgentState):
